## Deriving covariates across the northern bobwhite range


In [1]:
from datetime import date
today = date.today()
import ee
import math
import geemap
import numpy as np
import pandas as pd
import os


# Initialize ee and authenticate 
#ee.Authenticate()
ee.Initialize()



# Important Information

## Script name: 

## Purpose of script:
This is a script for deriving covariates across the northern bobwhite range for use in the construction of a species distribution model.
## Author: 
Patrick Freeman (CSP)
## Date Created: 
04/25/23
## Date last modified:
07/19/23
## Email: 
patrick[at]csp-inc.org
## ---------------------------
## Notes:

## ---------------------------

In [ ]:
#install geemap module as needed
#!pip install geemap 

# Write utility functions

In [2]:
# FUNCTIONS

# Focal mean
def focal_mean(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.mean()).rename(new_names)

# Focal median
def focal_median(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.median())
    
# Focal SD
def focal_sd(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.stdDev())

# Focal sum
def focal_sum(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.sum())

# Focal count
def focal_count(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.count())
    
# Percent cover
def percent_cov(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    isum = focal_sum(image, radius, unit)
    icount = focal_count(image, radius, unit)
    return isum.divide(icount).rename(new_names)

#----------------------------------------
# FUNCTIONS

# Focal mean
def focal_mean(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.mean()).rename(new_names)

# Focal median
def focal_median(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.median())
    
# Focal SD
def focal_sd(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit),
                                    reducer = ee.Reducer.stdDev())

# Focal sum
def focal_sum(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.sum())

# Focal count
def focal_count(image, radius, unit):
    return image.reduceNeighborhood(kernel = ee.Kernel.circle(radius, unit, False),
                                    reducer = ee.Reducer.count())
    
# Percent cover
def percent_cov(image, radius, unit, name):
    names = image.bandNames().getInfo()
    new_names = [s + name for s in names]
    isum = focal_sum(image, radius, unit)
    icount = focal_count(image, radius, unit)
    return isum.divide(icount).rename(new_names)

# Vector ruggedness measure
def compute_vrm(slope_img, aspect_img, radius, units):
    slope_sine = slope_img.sin()
    x_sum_sq = focal_sum(slope_sine.multiply(aspect_img.sin()), radius, units).pow(2)
    y_sum_sq = focal_sum(slope_sine.multiply(aspect_img.cos()), radius, units).pow(2)
    z_sum_sq = focal_sum(slope_img.cos(), radius, units).pow(2)
    n = focal_sum(ee.Image(1), radius, units)
    r = x_sum_sq.add(y_sum_sq).add(z_sum_sq).sqrt()
    vrm_img = ee.Image(1).subtract(r.divide(n))
    return vrm_img

def toFloat(img):
    return img.float()
#----------------------------------------

def toFloat(img):
    return img.float()

# Set aoi, spatial scale and projection of export, and smoothing parameters

In [3]:
## Bring in buffered range map as 'region' 
region = ee.FeatureCollection('projects/GEE_CSP/pf-bobwhite/bobwhite_model_states')
geometry = ee.Feature(ee.FeatureCollection(region).first())
conus_geom = ee.FeatureCollection("projects/GEE_CSP/thirty-by-thirty/aoi_conus")
conus_img = ee.Image("projects/GEE_CSP/thirty-by-thirty/aoi_conus_mask")

# export scale and projection
scale = 270
projection = ee.Projection('EPSG:5070') # stand-in for now. Figure out best projection to use 

### Load sampling grid
grid_5km = ee.FeatureCollection("projects/GEE_CSP/pf-bobwhite/grid_5km")

# Define a function to extract the centroid of a feature and create a new feature with that centroid as its geometry
def get_centroid(feature):
    keepProperties = ['grid_id_5k']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)

# Map the get_centroid function over the FeatureCollection to create a new FeatureCollection containing just the centroids
centroids = grid_5km.map(get_centroid)

# Choose radii for summarizing covariates
rad_small = 2500
name_small = "_5km"

## Plot to check 
#Map = geemap.Map(center=(40, -100), zoom=4)
#Map.addLayer(geometry, {}, "Model states", True)
#Map


# CSP-derived land use intensity layers related to agriculture, transportation, urban development, and energy infrastructure

In [4]:
lui = ee.Image("projects/GEE_CSP/aft-connectivity/Land-use-intensity-multiband-focal-sp-250m-20220123")

### Get the band names as a check 
lui_names = lui.bandNames()
print('lui Band names:', lui_names.getInfo())  # ee.List of band names
lui_focal_means_small = focal_mean(lui, rad_small, "meters", name_small).updateMask(conus_img).clip(geometry)

### Get the band names as a check 
lui_focal_means_names = lui_focal_means_small.bandNames()
print('lui_focal_means Band names:', lui_focal_means_names.getInfo())  # ee.List of band names

lui Band names: ['Ag', 'Urban', 'Transport', 'Energy']
lui_focal_means Band names: ['Ag_5km', 'Urban_5km', 'Transport_5km', 'Energy_5km']


In [17]:
def extract_lui_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = lui_focal_means_small.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=250)

  # Set the values as properties of the feature
  return feature.set(values)

# Map the extract_values function over the feature collection
lui_results = centroids.map(extract_lui_values)


# Export the feature collection as a CSV file
task = ee.batch.Export.table.toDrive(
    collection=lui_results,
    description='lui-export',
    folder='GEE-exports',
    fileNamePrefix='LUI_5km_smooth_230719',
    fileFormat='CSV')
task.start()

# Calculate percent cover of row crop from NLCD 

In [5]:
### Load all NLCD layers from data release 
nlcd_all = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

### The collection contains images for multiple years and regions in the USA.
print('Products:', nlcd_all.aggregate_array('system:index').getInfo())

nlcd_16 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").filter(ee.Filter.eq('system:index', '2016')).first().select('landcover')
nlcd_19 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").filter(ee.Filter.eq('system:index', '2019')).first().select('landcover')

ag_16 = ee.Image(0).where(nlcd_16.eq(82), 1).rename('rowcrop_16')
ag_19 = ee.Image(0).where(nlcd_19.eq(82), 1).rename('rowcrop_19')

ag_all = ee.Image([ag_16, ag_19])

rowcrop_pcov_all = percent_cov(ag_all, rad_small, 'meters', '_pcov' + name_small).clip(geometry)

rowcrop_pcov_all_band_names = rowcrop_pcov_all.bandNames()
print('rowcrop_pcov_all_band_names Band names:', rowcrop_pcov_all_band_names.getInfo())  # ee.List of band names

rowcrop_pcov_avg = rowcrop_pcov_all.reduce(ee.Reducer.mean()).rename(['NLCD_1619_mean_rowcropPcov'])




Products: ['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019']
rowcrop_pcov_all_band_names Band names: ['rowcrop_16_pcov_5km', 'rowcrop_19_pcov_5km']


### Calculate proportional cover of pasture from NLCD 

In [6]:

pasture_16 = ee.Image(0).where(nlcd_16.eq(81), 1).rename('pasture_16')
pasture_19 = ee.Image(0).where(nlcd_19.eq(81), 1).rename('pasture_19')

pasture_all = ee.Image([pasture_16, pasture_19])

pasture_pcov_all = percent_cov(pasture_all, rad_small, 'meters', '_pcov' + name_small).clip(geometry)

pasture_pcov_all_band_names = pasture_pcov_all.bandNames()
print('pasture_pcov_all_band_names Band names:', pasture_pcov_all_band_names.getInfo())  # ee.List of band names

pasture_pcov_avg = pasture_pcov_all.reduce(ee.Reducer.mean()).rename(['NLCD_1619_mean_pasturePcov'])

pasture_pcov_all_band_names Band names: ['pasture_16_pcov_5km', 'pasture_19_pcov_5km']


# Export ag proportional cover as needed

In [21]:
#### Write function to perform the raster extraction (for each raster) -- importantly set scale to the native resolution of whatever raster you're extracting from (e.g. 30m for RAP, 250m for LUI, etc.)
def extract_agpcov_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = rowcrop_pcov_avg.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=30)

  # Set the values as properties of the feature
  return feature.set(values)

def extract_pasturepcov_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = pasture_pcov_avg.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=30)

  # Set the values as properties of the feature
  return feature.set(values)

# Map the extract_values function over the feature collection
agpcov_results = centroids.map(extract_agpcov_values)
pasturepcov_results = centroids.map(extract_pasturepcov_values)

# Export the feature collection as a CSV file
task1 = ee.batch.Export.table.toDrive(
    collection=agpcov_results,
    description='RowcropPCOV-export',
    folder='GEE-exports',
    fileNamePrefix='RowcropPCov_5km_1619_230719',
    fileFormat='CSV')

# Export the feature collection as a CSV file
task2 = ee.batch.Export.table.toDrive(
    collection=pasturepcov_results,
    description='PasturePCOV-export',
    folder='GEE-exports',
    fileNamePrefix='PasturePCov_5km_1619_230719',
    fileFormat='CSV')

task1.start()
task2.start()


# RAP Proportional Cover - first calculate multi-year averages and apply smoothing within 5km smoothing windows 

In [5]:
geometry = ee.Feature(ee.FeatureCollection("projects/GEE_CSP/pf-bobwhite/bobwhite_model_states").first());
##---------- Define the years that you want to export --------------
##---------- End year is inclusive in this case  ------------------
yearStart = 2018
yearEnd = 2021

## -------------- Define the plant functional types (PFTs) that you want to export --------------
## PFTs are "AFGC" (Annual forb and grass cover), "BG" (bare ground), "LTR" (litter), 
## "PFGC" (perennial forb and grass cover), "SHR" (shrub cover), and "TREE" (tree cover)
## Select Annual forb and grass cover, perennial forb and grass cover, shrub cover, and tree cover 
PFTs = ee.List(['AFG', 'PFG', 'SHR', 'TRE', 'BGR'])

cover = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
## ------------- Select the PFTs for processing as defined by User  --------------
cover_toExport = cover.select(PFTs).filter(ee.Filter.inList('year', ee.List([2018, 2019, 2020, 2021]))).toBands()

band_names = cover_toExport.bandNames()
print('band_names:', band_names.getInfo())  # ee.List of band names



band_names: ['2018_AFG', '2018_PFG', '2018_SHR', '2018_TRE', '2018_BGR', '2019_AFG', '2019_PFG', '2019_SHR', '2019_TRE', '2019_BGR', '2020_AFG', '2020_PFG', '2020_SHR', '2020_TRE', '2020_BGR', '2021_AFG', '2021_PFG', '2021_SHR', '2021_TRE', '2021_BGR']


### Get mean value of each individual cover type across all years 

In [6]:

# Define string to match
string_to_match = "_TRE"
# Get the band names from the image
band_names = cover_toExport.bandNames()
# Filter the bands to select only the ones that contain the partial string
tree_bands = band_names.filter(ee.Filter.stringContains("item", string_to_match))
# Select the TREE COVER bands from the image, calculate the multi-year mean, and apply the focal smooth operation
tree_img = cover_toExport.select(tree_bands).reduce(ee.Reducer.mean()).rename(['RAP_TRE_1821_mean'])

tree_mean_smooth = focal_mean(tree_img, rad_small, "meters", name_small).clip(geometry)
tree_mean_smooth_band_names = tree_mean_smooth.bandNames()
print('tree_mean_smooth Band names:', tree_mean_smooth_band_names.getInfo())  # ee.List of band names

tree_mean_smooth Band names: ['RAP_TRE_1821_mean_5km']


In [7]:

# Select the SHRUB COVER bands from the image
string_to_match = "_SHR"
# Filter the bands to select only the ones that contain the partial string
shrub_bands = band_names.filter(ee.Filter.stringContains("item", string_to_match))
# Select the SHRUB COVER bands from the image
shrub_img = cover_toExport.select(shrub_bands).reduce(ee.Reducer.mean()).rename(['RAP_SHR_1821_mean'])

shrub_mean_smooth = focal_mean(shrub_img, rad_small, "meters", name_small).clip(geometry)
shrub_mean_smooth_band_names = shrub_mean_smooth.bandNames()
print('shrub_mean_smooth Band names:', shrub_mean_smooth_band_names.getInfo())  # ee.List of band names


shrub_mean_smooth Band names: ['RAP_SHR_1821_mean_5km']


In [8]:

# Select the ANNUAL FORB AND GRASS COVER bands from the image
string_to_match = "_AFG"
# Filter the bands to select only the ones that contain the partial string
afg_bands = band_names.filter(ee.Filter.stringContains("item", string_to_match))
afg_img = cover_toExport.select(afg_bands).reduce(ee.Reducer.mean()).rename(['RAP_AFG_1821_mean'])

afg_mean_smooth = focal_mean(afg_img, rad_small, "meters", name_small).clip(geometry)
afg_mean_smooth_band_names = afg_mean_smooth.bandNames()
print('afg_mean_smooth Band names:', afg_mean_smooth_band_names.getInfo())  # ee.List of band names


afg_mean_smooth Band names: ['RAP_AFG_1821_mean_5km']


In [9]:

# Select the PERENNIAL FORB AND GRASS COVER bands from the image
string_to_match = "_PFG"
# Filter the bands to select only the ones that contain the partial string
pfg_bands = band_names.filter(ee.Filter.stringContains("item", string_to_match))
# Select the ANNUAL FORB AND GRASS COVER bands from the image
pfg_img = cover_toExport.select(pfg_bands).reduce(ee.Reducer.mean()).rename(['RAP_PFG_1821_mean'])

pfg_mean_smooth = focal_mean(pfg_img, rad_small, "meters", name_small).clip(geometry)
pfg_mean_smooth_band_names = pfg_mean_smooth.bandNames()
print('pfg_mean_smooth Band names:', pfg_mean_smooth_band_names.getInfo())  # ee.List of band names


pfg_mean_smooth Band names: ['RAP_PFG_1821_mean_5km']


In [10]:

# Select the BARE GROUND COVER bands from the image
string_to_match = "_BGR"
# Filter the bands to select only the ones that contain the partial string
bgr_bands = band_names.filter(ee.Filter.stringContains("item", string_to_match))
# Select the ANNUAL FORB AND GRASS COVER bands from the image
bgr_img = cover_toExport.select(bgr_bands).reduce(ee.Reducer.mean()).rename(['RAP_BGR_1821_mean'])

bgr_mean_smooth = focal_mean(bgr_img, rad_small, "meters", name_small).clip(geometry)
bgr_mean_smooth_band_names = bgr_mean_smooth.bandNames()
print('bgr_mean_smooth Band names:', bgr_mean_smooth_band_names.getInfo())  # ee.List of band names

bgr_mean_smooth Band names: ['RAP_BGR_1821_mean_5km']


### Extract Smoothed RAP Values to Cell Centroids

In [13]:
final_rap_img = ee.Image([bgr_mean_smooth, pfg_mean_smooth, afg_mean_smooth, shrub_mean_smooth, tree_mean_smooth])

#### Write function to perform the raster extraction (for each raster) -- importantly set scale to the native resolution of whatever raster you're extracting from (e.g. 30m for RAP, 250m for LUI, etc.)
def extract_rap_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = final_rap_img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=30)

  # Set the values as properties of the feature
  return feature.set(values)

# Map the extract_values function over the feature collection
rap_results = centroids.map(extract_rap_values)


# Export the feature collection as a CSV file
task = ee.batch.Export.table.toDrive(
    collection=rap_results,
    description='RAP-export',
    folder='GEE-exports',
    fileNamePrefix='RAP_1821_5km_smooth_230719',
    fileFormat='CSV')
task.start()


# Collect the raw (unsmoothed) multi-year average RAP cover images into a multi-band image for gradient analysis  - export

In [14]:

RAP_unsmoothed = ee.Image([tree_img, shrub_img, afg_img, pfg_img, bgr_img ])

# Convert all to float-32
RAP_unsmoothed = RAP_unsmoothed.float()
scale=270
task1 = ee.batch.Export.image.toDrive(image = RAP_unsmoothed,
                                     folder = 'GEE-exports',
                                     description = 'RAP-1821-mean-unsmoothed' + str(scale) + "m",
                                     scale = scale,
                                     region = geometry.geometry(),
                                     maxPixels = 1e13,
                                     crs = "EPSG:5070")
task1.start()

# Climate covariates from Daymet

In [15]:
### Load daymet dataset 
#daymet_16 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2016-01-01', '2016-12-31'))
#daymet_17 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2017-01-01', '2017-12-31'))
daymet_18 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2018-01-01', '2018-12-31'))
daymet_19 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2019-01-01', '2019-12-31'))
daymet_20 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2020-01-01', '2020-12-31'))
daymet_21 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2021-01-01', '2021-12-31'))

daymet_1821 = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filter(ee.Filter.date('2018-01-01', '2021-12-31'))
tmax_1821 = daymet_1821.select("tmax").mean().rename(['tmax_1821_mean'])
tmin_1821 = daymet_1821.select("tmin").mean().rename(['tmin_1821_mean'])
prcp_1821 = daymet_1821.select("prcp").mean().rename(['prcp_1821_mean'])
swe_1821 = daymet_1821.select("swe").mean().rename(['swe_1821_mean'])

climate_1821 = ee.Image([tmax_1821, tmin_1821, prcp_1821, swe_1821])
climate_1821_small = focal_mean(climate_1821, rad_small, "meters", name_small).updateMask(conus_img).clip(geometry)


In [20]:
def extract_climate_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = climate_1821_small.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=1000)

  # Set the values as properties of the feature
  return feature.set(values)

# Map the extract_values function over the feature collection
climate_results = centroids.map(extract_climate_values)


# Export the feature collection as a CSV file
task = ee.batch.Export.table.toDrive(
    collection=climate_results,
    description='climate-export',
    folder='GEE-exports',
    fileNamePrefix='climate_5km_smooth_230719',
    fileFormat='CSV')
task.start()

In [16]:

# Convert all to float-32
climate_1821_small = climate_1821_small.float()
scale=1000
task1 = ee.batch.Export.image.toDrive(image = climate_1821_small,
                                     folder = 'GEE-exports',
                                     description = 'climate-1821-mean-smoothed' + str(scale) + "m" ,
                                     scale = scale,
                                     region = geometry.geometry(),
                                     maxPixels = 1e13,
                                     crs = "EPSG:5070")
task1.start()

### Terrain and Topography variables

In [17]:
#---------------------------------------------
# TERRAIN VARIABLES
# Create some topographic viables
dsm = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2") # Digital surface model. Native res 30m
projElev = dsm.first().select(0).projection()
elev = dsm.select("DSM").mosaic().setDefaultProjection(projElev).rename('elevation') # Elevation
slope = ee.Terrain.slope(elev).rename('slope') # Slope
aspect = ee.Terrain.aspect(elev).rename('aspect') # Aspect
# Get vector ruggedness metric
window_radius = 1000
slopeRad = slope.multiply(ee.Number(math.pi).divide(180))
aspectRad = slope.multiply(ee.Number(math.pi).divide(180))
#vrm = compute_vrm(slopeRad, aspectRad, window_radius, "meters").rename('vrm')
chili = ee.Image("CSP/ERGo/1_0/US/CHILI").rename('CHILI')

# Combine and smooth
terrain_all = ee.Image([elev, slope, aspect, chili])
terrain_small = focal_mean(terrain_all, rad_small, "meters", name_small)




### Extract terrain variables to grid cell centroids

In [18]:
#### Write function to perform the raster extraction (for each raster) -- importantly set scale to the native resolution of whatever raster you're extracting from (e.g. 30m for RAP, 250m for LUI, etc.)
def extract_terrain_values(feature):
  # Get the geometry of the feature
  geometry = feature.geometry()

  # Extract the raster values for the feature
  values = terrain_small.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=geometry,
      scale=30)

  # Set the values as properties of the feature
  return feature.set(values)

# Map the extract_values function over the feature collection
terrain_results = centroids.map(extract_terrain_values)


# Export the feature collection as a CSV file
task = ee.batch.Export.table.toDrive(
    collection=terrain_results,
    description='Terrain-export-noVRM',
    folder='GEE-exports',
    fileNamePrefix='terrain_5km_smooth_230719',
    fileFormat='CSV')
task.start()

### Parking lot 

In [27]:
# Convert all to float-32
scale=30
terrain_raster = terrain_small.float()
task1 = ee.batch.Export.image.toDrive(image = terrain_raster,
                                     folder = 'GEE-exports',
                                     description = 'terrain-mean-smoothed' + str(scale) + "m",
                                     scale = scale,
                                     region = geometry.geometry(),
                                     maxPixels = 1e13,
                                     crs = "EPSG:5070")
task1.start()